In [ ]:
pip install yfinance

In [31]:
import yfinance as yf
import numpy as np
import networkx as nx

def stock_volatility(symbol,
                     start_date="2020-02-14",
                     end_date="2020-04-14",
                     interval="1d"):
    df = yf.download(symbol,
                     start=start_date,
                     end=end_date,
                     interval=interval,
                     progress=False)
    daily_range = df["High"] - df["Low"]
    return daily_range.diff().abs().sum()

def stock_marketcap(symbols):
    """
    Return a list of market-caps (in billions USD) for each ticker in `symbols`,
    using yfinance’s .info["marketCap"].
    """
    caps = []
    for sym in symbols:
        info = yf.Ticker(sym).info
        mc = info.get("marketCap", None)
        if mc is None:
            caps.append(np.nan)
        else:
            # convert raw marketCap (e.g. 450_000_000_000) to billions
            caps.append(mc / 1e9)
    return caps

def stock_coefficient(market_caps):
    m = len(market_caps)
    M = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            M[i, j] = market_caps[i] / market_caps[j]
    return M

def stock_graph_coefficient(coeff_matrix):
    A = np.ones_like(coeff_matrix) - np.eye(coeff_matrix.shape[0])
    return A * coeff_matrix

def stock_volatility_matrix(symbols, start_date, end_date, interval):
    vols = [stock_volatility(sym, start_date, end_date, interval)
            for sym in symbols]
    return np.diag(vols), vols

In [32]:
def Stock_Ranking_Algo_with_Convolution_and_PageRank(symbols,
                                                     start_date="2020-02-14",
                                                     end_date="2020-04-14",
                                                     interval="1d"):
    # Step 1: volatility
    V_mat, vols = stock_volatility_matrix(symbols, start_date, end_date, interval)
    # Step 2: market caps via yfinance
    caps = stock_marketcap(symbols)
    # Step 3: pairwise cap ratios
    coeff = stock_coefficient(caps)
    # Step 4: build weighted adjacency (no self-loops)
    A = stock_graph_coefficient(coeff)
    # Step 5: convolution = A · vols
    conv_vol = A.dot(np.array(vols))
    # Step 6: build directed graph for PageRank
    G = nx.DiGraph()
    n = len(symbols)
    for i in range(n):
        for j in range(n):
            if i != j:
                G.add_edge(symbols[j], symbols[i], weight=A[i, j])
    # personalization = normalized convolved vols
    p = {symbols[i]: conv_vol[i].item() for i in range(n)}
    total = sum(p.values())
    p = {k: v/total for k, v in p.items()}


    # Step 7: run PageRank
    pr = nx.pagerank(G, weight="weight", personalization=p)
    # Return tickers sorted by score descending
    return sorted(pr.items(), key=lambda x: x[1], reverse=True)

In [36]:
tickers = ["TSLA", "GM", "F", "VWAGY", "TM"]
Stock_Ranking_Algo_with_Convolution_and_PageRank(tickers, start_date="2025-01-01", end_date="2025-06-29",interval="1d")

/tmp/ipython-input-31-4265138506.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol,
/tmp/ipython-input-31-4265138506.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol,
/tmp/ipython-input-31-4265138506.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol,
/tmp/ipython-input-31-4265138506.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol,
/tmp/ipython-input-31-4265138506.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol,


[('TSLA', 0.47711755573292014),
 ('TM', 0.30518980425643516),
 ('VWAGY', 0.08110645540094337),
 ('GM', 0.07046438225267726),
 ('F', 0.06612180235702375)]